In [1]:
import os
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from utl2 import dataset, ToTensor
from cnn_mlp import CNN_MLP
from cnn_lstm import CNN_LSTM
from resnet18 import Resnet18_MLP
from cnn_logical_v1 import CNN_Logical

In [2]:
class args():
    model='CNN_Logical'
    epochs=400
    batch_size=32
    seed=12345
    dataset = 'iRaven'
    device = 0
    load_workers = 16
    resume = False
    path = "/common/users/pv217/pritish_data"
    save = "/common/users/pv217/model_checkpoints/full_cnn_logical_all_aug_10aug/"
    img_size = 80
    lr = 1e-4
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-8
    meta_alpha = 0.0
    meta_beta = 0.0
    cuda = torch.cuda.is_available()    

In [3]:
args.cuda = torch.cuda.is_available()
torch.cuda.set_device(args.device)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [4]:
data_path = "/common/users/pv217/pritish_data"

In [5]:
train = dataset(data_path, "train", args.img_size, transform=transforms.Compose([ToTensor()]),shuffle=True,
                rotate=True,vertical_flip=True, vertical_roll = True,
                horizontal_flip = True, horizontal_roll= True, max_rotate_angle = 180
               )
valid = dataset(data_path, "val", args.img_size, transform=transforms.Compose([ToTensor()]))
test = dataset(data_path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))

In [6]:
trainloader = DataLoader(train, batch_size=args.batch_size, shuffle=True, num_workers=16)
validloader = DataLoader(valid, batch_size=args.batch_size, shuffle=False, num_workers=16)
testloader = DataLoader(test, batch_size=args.batch_size, shuffle=False, num_workers=16)

In [7]:
train.__getitem__(1)[2].shape

torch.Size([9])

In [8]:
train.__getitem__(1)[2].sum()

tensor(7.)

In [9]:
if args.model == "CNN_MLP":
    model = CNN_MLP(args)
elif args.model == "CNN_LSTM":
    model = CNN_LSTM(args)
elif args.model == "Resnet18_MLP":
    model = Resnet18_MLP(args)
elif args.model == "CNN_Logical":
    model = CNN_Logical(args)

In [10]:
if args.resume:
    model.load_model(args.save, 0)
    print('Loaded model')
if args.cuda:
    model = model.cuda()

In [11]:
def train(epoch):
    model.train()
    train_loss = 0
    accuracy = 0

    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(trainloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
#             meta_structure = meta_structure.cuda()
#             embedding = embedding.cuda()
#             indicator = indicator.cuda()
        loss, acc = model.train_(image, target, meta_target)
        #print('Train: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}.'.format(epoch, batch_idx, loss, acc))
        loss_all += loss
        acc_all += acc
    print(epoch)
    if counter > 0:
        print("Avg Training Loss: {:.6f}".format(loss_all/float(counter)))
    return loss_all/float(counter), acc_all/float(counter)


In [12]:
def validate(epoch):
    model.eval()
    val_loss = 0
    accuracy = 0

    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(validloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
#             meta_structure = meta_structure.cuda()
#             embedding = embedding.cuda()
#             indicator = indicator.cuda()
        loss, acc = model.validate_(image, target, meta_target)
        # print('Validate: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}.'.format(epoch, batch_idx, loss, acc)) 
        loss_all += loss
        acc_all += acc
    if counter > 0:
        print("Total Validation Loss: {:.6f}, Acc: {:.4f}".format(loss_all/float(counter), acc_all/float(counter)))
    return loss_all/float(counter), acc_all/float(counter)

In [13]:
def test(epoch):
    model.eval()
    accuracy = 0

    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(testloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
#             meta_structure = meta_structure.cuda()
#             embedding = embedding.cuda()
#             indicator = indicator.cuda()
        acc = model.test_(image, target, meta_target)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        acc_all += acc
    if counter > 0:
        print("Total Testing Acc: {:.4f}".format(acc_all / float(counter)))
    return acc_all/float(counter)

In [ ]:
epoch_lst = []
train_loss_lst = []
train_acc_lst = []
val_loss_lst = []
val_acc_lst = []
test_acc_lst = []

for epoch in range(0, args.epochs):
    train_loss, train_acc = train(epoch)
    val_loss, val_acc = validate(epoch)
    test_acc = test(epoch)
    epoch_lst.append(epoch)
    train_loss_lst.append(train_loss)
    train_acc_lst.append(train_acc)
    val_loss_lst.append(val_loss)
    val_acc_lst.append(val_acc)
    test_acc_lst.append(test_acc)
    model.save_model(args.save, epoch, val_acc, val_loss)
    
    

0
Avg Training Loss: 2.461007
Total Validation Loss: 2.203331, Acc: 12.4857
Total Testing Acc: 12.8853
1
Avg Training Loss: 2.189420
Total Validation Loss: 2.141751, Acc: 12.0576
Total Testing Acc: 12.4572
2
Avg Training Loss: 2.150948
Total Validation Loss: 2.126423, Acc: 12.4715
Total Testing Acc: 12.6998
3
Avg Training Loss: 2.135686
Total Validation Loss: 2.112856, Acc: 12.8139
Total Testing Acc: 12.5856
4
Avg Training Loss: 2.124443
Total Validation Loss: 2.113324, Acc: 13.2063
Total Testing Acc: 12.8995
5
Avg Training Loss: 2.115853
Total Validation Loss: 2.109184, Acc: 13.0708
Total Testing Acc: 13.1064
6
Avg Training Loss: 2.113154
Total Validation Loss: 2.096251, Acc: 12.8853
Total Testing Acc: 12.9352
7
Avg Training Loss: 2.110111
Total Validation Loss: 2.095651, Acc: 13.3990
Total Testing Acc: 13.4275
8
Avg Training Loss: 2.103269
Total Validation Loss: 2.089283, Acc: 12.5999
Total Testing Acc: 12.7497
9
Avg Training Loss: 2.105401
Total Validation Loss: 2.089493, Acc: 12.54

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
training_stats = pd.DataFrame({'epoch':epoch_lst,
 'training_loss':train_loss_lst,
 'training_accuracy':train_acc_lst,
 'validation_loss':val_loss_lst,
 'validation_accuracy':val_acc_lst,
 'test_accuracy':test_acc_lst
})

In [ ]:
training_stats = training_stats.set_index('epoch')

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})

In [ ]:
sns.lineplot(data=training_stats[['training_loss','validation_loss']])

In [ ]:
sns.lineplot(data=training_stats[['training_accuracy','validation_accuracy','test_accuracy']])

#### Checking Model performance on each configuration  

In [ ]:
def validate():
    model.eval()
    val_loss = 0
    accuracy = 0
    epoch = 1
    loss_all = 0.0
    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(validloader):
        counter += 1
#         print(counter)
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
#             meta_structure = meta_structure.cuda()
        loss, acc = model.validate_(image, target, meta_target)
#         print('Validate: Epoch:{}, Batch:{}, Loss:{:.6f}, Acc:{:.4f}.'.format(epoch, batch_idx, loss, acc)) 
        loss_all += loss
        acc_all += acc
#         print(counter)
    if counter >0:
        print("Total Validation Loss: {:.6f}, Acc: {:.4f}".format(loss_all/float(counter), acc_all/float(counter)))
    return  acc_all/float(counter)

def test():
    model.eval()
    accuracy = 0
    epoch = 1
    acc_all = 0.0
    counter = 0
    for batch_idx, (image, target, meta_target) in enumerate(testloader):
        counter += 1
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
#             meta_structure = meta_structure.cuda()
#             embedding = embedding.cuda()
#             indicator = indicator.cuda()
        acc = model.test_(image, target, meta_target)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        acc_all += acc
    if counter > 0:
        print("Total Testing Acc: {:.4f}".format(acc_all / float(counter)))
    return acc_all/float(counter)

In [ ]:
config_lst = []
config_val_acc_list = []
config_test_acc_list = []
for i in ['/center_single/','/distribute_four/','/distribute_nine/',
          '/in_center_single_out_center_single/','/in_distribute_four_out_center_single/', 
          '/left_center_single_right_center_single/','/up_center_single_down_center_single/']:
    valid_dt = dataset(data_path, "val", args.img_size, transform=transforms.Compose([ToTensor()]))
    test_dt = dataset(data_path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
    valid_dt.file_names = [x for x in valid_dt.file_names if i in x]
    test_dt.file_names = [x for x in test_dt.file_names if i in x]
    validloader = DataLoader(valid_dt, batch_size=args.batch_size, shuffle=False, num_workers=16)
    testloader = DataLoader(test_dt, batch_size=args.batch_size, shuffle=False, num_workers=16)
    config_lst.append(i[1:-1])
    config_val_acc_list.append(validate())
    config_test_acc_list.append(test())
    
    

In [ ]:
pd.DataFrame({'configuration':config_lst,'validation_accuracy':config_val_acc_list, 'test_accuracy':config_test_acc_list})

In [ ]:
sns.barplot(x="configuration", y="test_accuracy", data=pd.DataFrame({'configuration':config_lst,'validation_accuracy':config_val_acc_list, 'test_accuracy':config_test_acc_list}))

In [ ]:
# sns.s pd.DataFrame({'configuration':config_lst,'validation_accuracy':config_val_acc_list, 'test_accuracy':config_test_acc_list}).set_index('configuration')

In [ ]:
for i in ['/center_single/','/distribute_four/','/in_distribute_four_out_center_single/', '/left_center_single_right_center_single/']:
    train_dt = dataset(data_path, "train", args.img_size, transform=transforms.Compose([ToTensor()]))
    valid_dt = dataset(data_path, "val", args.img_size, transform=transforms.Compose([ToTensor()]))
    test_dt = dataset(data_path, "test", args.img_size, transform=transforms.Compose([ToTensor()]))
    train_dt.file_names = [x for x in train_dt.file_names if i in x]
    valid_dt.file_names = [x for x in valid_dt.file_names if i in x]
    test_dt.file_names = [x for x in test_dt.file_names if i in x]
    print(i,len(set(train_dt.file_names)),len(set(valid_dt.file_names)),len(set(test_dt.file_names)))


In [ ]:
training_stats.tail(1)

In [ ]:
from utl2 import dataset, ToTensor

In [ ]:
# data_path_t = '/common/home/pv217/Downloads/Data-Color/'
test_dt = dataset(data_path, "test", args.img_size, transform=transforms.Compose([ToTensor()]),matrix=True)

test_dt.file_names = [x for x in test_dt.file_names if '/center_single/' in x]
testloader = DataLoader(test_dt, batch_size=args.batch_size, shuffle=False, num_workers=16)


In [ ]:
def test_ra_pairs_(model, image, target, ra_pair_matrix):
        with torch.no_grad():
            output = model(image)
        pred = output[0].data.max(1)[1]
        correct = pred.eq(target.data).cpu().sum().numpy()
        ra_pair_matrix_correct = pred.eq(target.data).cpu().numpy().reshape(-1,1,1)*ra_pair_matrix.numpy()
        accuracy = correct * 100.0 / target.size()[0]
        return accuracy, ra_pair_matrix_correct.sum(axis=0)

In [ ]:
def test_ra_pairs():
#     model.eval()
    accuracy = 0
    epoch = 1
    acc_all = 0.0
    counter = 0
    data_ra_pairs = np.zeros((4,5))
    target_ra_pairs = np.zeros((4,5))
    for batch_idx, (image, target, meta_target,rule_attribute_matrix) in enumerate(testloader):
#         print(rule_attribute_matrix[10])
#         a = meta_matrix
        counter += 1
        data_ra_pairs += rule_attribute_matrix.sum(axis=0).numpy()
        
        if args.cuda:
            image = image.cuda()
            target = target.cuda()
            meta_target = meta_target.cuda()
#             meta_structure = meta_structure.cuda()
#             embedding = embedding.cuda()
#             indicator = indicator.cuda()
        
        acc,acc_ra_pairs = test_ra_pairs_(model,image, target, rule_attribute_matrix)
        # print('Test: Epoch:{}, Batch:{}, Acc:{:.4f}.'.format(epoch, batch_idx, acc))  
        target_ra_pairs +=acc_ra_pairs
        acc_all += acc
    if counter > 0:
        print("Total Testing Acc: {:.4f}".format(acc_all / float(counter)))
#     return rule_attribute_matrix
    return acc_all/float(counter),data_ra_pairs,target_ra_pairs

In [ ]:
acc, data_ra_matrix, accurate_ra_matrix = test_ra_pairs()

In [ ]:
accurate_ra_matrix

In [ ]:
data_ra_matrix

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns
%matplotlib inline
accuracy = accurate_ra_matrix/data_ra_matrix
Mask = np.zeros(np.shape(accuracy))
Mask[accuracy<=0.01] = 1
_accuracy = accuracy*100.0
midpoint = (_accuracy.max() - _accuracy.min()) / 2
# plot the heatmap
# plt.rcParams['font.size'] = 12
plt.rcParams["font.family"] = "Times New Roman"
fig, ax = plt.subplots(1)
sns.heatmap(_accuracy, 
            mask=Mask,
            vmin=0.0, vmax=90.0,
            #center = 45.0,
            cmap="coolwarm",
            square=True,
            robust=True,
            annot=True, fmt=".2f",annot_kws={'size':12},
            cbar=False,
        yticklabels=['Constant', 'Progression', 'Arithmetic', 'Distribute Three'],
        xticklabels=['Num', 'Pos', 'Type', 'Size', 'Color'])
# plt.xlabel('Attributes', fontsize=16)
# plt.ylabel('Rules', fontsize=16)
plt.savefig(f"heatmap_centersingle_i-raven.pdf", bbox_inches='tight', pad_inches=0)
# plt.text(5,12.3, "I-RAVEN", fontsize = 95, color='Black', fontstyle='italic')


In [ ]:
# data_path_t = '/common/home/pv217/Downloads/Data-Color/'
test_dt = dataset(data_path, "test", args.img_size, transform=transforms.Compose([ToTensor()]),matrix=True)

# test_dt.file_names = [x for x in test_dt.file_names if '/center_single/' in x]
testloader = DataLoader(test_dt, batch_size=args.batch_size, shuffle=False, num_workers=16)


In [ ]:
acc, data_ra_matrix, accurate_ra_matrix = test_ra_pairs()

In [ ]:
import seaborn as sns
%matplotlib inline
accuracy = accurate_ra_matrix/data_ra_matrix
Mask = np.zeros(np.shape(accuracy))
Mask[accuracy<=0.01] = 1
_accuracy = accuracy*100.0
midpoint = (_accuracy.max() - _accuracy.min()) / 2
# plot the heatmap
# plt.rcParams['font.size'] = 12
plt.rcParams["font.family"] = "Times New Roman"
fig, ax = plt.subplots(1)
sns.heatmap(_accuracy, 
            mask=Mask,
            vmin=0.0, vmax=55.0,
            #center = 45.0,
            cmap="coolwarm",
            square=True,
            robust=True,
            annot=True, fmt=".2f",annot_kws={'size':12},
            cbar=False,
        yticklabels=['Constant', 'Progression', 'Arithmetic', 'Distribute Three'],
        xticklabels=['Num', 'Pos', 'Type', 'Size', 'Color'])
# plt.xlabel('Attributes', fontsize=16)
# plt.ylabel('Rules', fontsize=16)
plt.savefig(f"heatmap_i-raven.pdf", bbox_inches='tight', pad_inches=0)
# plt.text(5,12.3, "I-RAVEN", fontsize = 95, color='Black', fontstyle='italic')
